In [43]:
from transformers import AutoTokenizer
from scipy.special import softmax
import os
import tweepy 
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()


In [38]:
import os
import tweepy 
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from datetime import datetime
from dotenv import load_dotenv

class api():
    def __init__(self,bearer) -> None:
        self.client = tweepy.Client(bearer_token=bearer)

    def scrape(self,query,num_tweets):
 
        # Creating DataFrame using pandas
        db = pd.DataFrame(columns=['date',
                                   'username',
                                   'author_id',
                                   'mentions',
                                   'hashtags',
                                   'location',
                                   'text'])

        tweets  = self.client.search_all_tweets(query=query, 
                                                tweet_fields = ['author_id','created_at','text','source','lang','geo'],
                                                user_fields = ['name','username','location','verified'],
                                                expansions = ['geo.place_id', 'author_id'],
                                                place_fields = ['country','country_code'] ,
                                                max_results=num_tweets)

        list_tweets = [tweet for tweet in tweets[0]]
        i = 1
        places = {p["id"]: p for p in tweets.includes['places']}
        for tweet in list_tweets:

                username = self.client.get_user(id = tweet.author_id).data.username
                raw_names = re.findall(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)",tweet.text)
                raw_names = [x.lower() for x in raw_names]
                if raw_names:
                        raw_names = list(dict.fromkeys(raw_names))

                        mentions = [x.id for x in self.client.get_users(usernames = raw_names).data] 
                else: 
                        mentions = []
                if places[tweet.geo['place_id']]:
                        location = places[tweet.geo['place_id']].full_name
                date = tweet.created_at
                hashtags = re.findall(r"#(\w+)", tweet.text)

                ith_tweet = [date,
                             username,
                             tweet.author_id,
                             mentions,
                             hashtags,
                             location ,
                             tweet.text]

                db.loc[len(db)] = ith_tweet
                i = i+1
        filename = 'scraped_tweets.csv'

        db.to_csv(filename,index = False)
        return db

load_dotenv()
from datetime import datetime
use = api(os.getenv('bearer'))
use.scrape('#Iran place_country:IR',10)

,date,username,author_id,mentions,hashtags,location,text
0,2022-12-05 18:03:58+00:00,RezayatHadis,1551256587129573378,[],"[Iran, pianist, مهسا_امینی]",République islamique d'Iran,#Iran\n#pianist \n#مهسا_امینی https://t.co/rc2...
1,2022-12-05 17:43:04+00:00,journalistsay,1385266495,[],"[Tehran, Iran, IranRevolution]",Sadeghiyeh Metro Station | ایستگاه مترو صادقیه,"Dec 5—#Tehran, #Iran \nThe protesters in the c..."
2,2022-12-05 11:27:49+00:00,salar__bagheri,1565399154121129986,[],"[اعتصابات_سراسری, ایران, Iran]",République islamique d'Iran,رییس محترم قوه قضاییه توصیه ای رو به والدین کر...
3,2022-12-04 23:25:24+00:00,alishahbaziart,1236046969140850688,[],"[مهسا_امینی, زن_زندگی_آزادی, ايران_آزاد, woman...",République islamique d'Iran,ظلم نمى ماند . . .\n.\n#مهسا_امینی #زن_زندگی_...
4,2022-12-04 21:24:48+00:00,marzprad,1580317621412511744,[],[Iran],République islamique d'Iran,Wanna know one of many truths related to the I...
5,2022-12-04 14:23:51+00:00,IrnaEnglish,813279088865644544,[],[Iran],République islamique d'Iran,#Iran FM spox slams crackdown on European prot...
6,2022-12-04 14:19:46+00:00,IrnaEnglish,813279088865644544,[],[Iran],République islamique d'Iran,Cmdr. says #Iran's Bushehr has progressed in l...
7,2022-12-04 13:48:58+00:00,sheykhian_h,1469389022271262729,[2359926157],"[Iran, ChinaUprising]",République islamique d'Iran,Blocking the streets in China is the same meth...
8,2022-12-04 08:39:12+00:00,babakhaghi1982,1410490384484950016,[],"[photography, streetphotography, outdoors, bab...",République islamique d'Iran,Rainy Afternoon \nhttps://t.co/YIkuOebAZ2\n202...
9,2022-12-04 08:18:40+00:00,AliGodFather92,830685854544908288,[],"[Europeans, protests, Iran, media]",République islamique d'Iran,Isn't it a bit strange for #Europeans that the...


In [41]:
class Sentiment():
    def __init__(self,english_model,english_tokenizer,farsi_model,farsi_tokenizer) -> None:
        self.english_model = english_model
        self.english_tokenizer = english_tokenizer
        self.farsi_model = farsi_model
        self.farsi_tokenizer = farsi_tokenizer
    def check_language(self,s): 
        s2 = re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', ' ',s).replace('\n',' ')
        s2 = re.sub(' +', ' ', s2)
        prop = len(s2)/len(s)
        if prop > .1: 
            input_ids = self.farsi_tokenizer.encode(s2, return_tensors="pt")
            res = self.farsi_model.generate(input_ids,max_length=300)
            output = self.farsi_tokenizer.batch_decode(res, skip_special_tokens=True)
            print(res)
            return output[0]
        else: 
            text = self.preprocess_english(s)
            encoded_input = self.english_tokenizer(text, return_tensors='pt')
            output = self.english_model(**encoded_input)
            scores = output[0][0].detach().numpy()
            scores = softmax(scores)
            ranking = np.argsort(scores)
            ranking = ranking[::-1]
            out = labels[np.argmax(ranking)]
            return out

    def analyze(self,df:pd.DataFrame,target_col='text',output_col = 'sentiment'):
        df[output_col] = df.text.apply(lambda x: self.check_language(x))
        return df 
    def preprocess_english(self,text):
        new_text = []
    
    
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)           
            

In [42]:

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer_english  = AutoTokenizer.from_pretrained(MODEL)
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
model_english = AutoModelForSequenceClassification.from_pretrained(MODEL)


tokenizer_farsi = AutoTokenizer.from_pretrained("persiannlp/mt5-base-parsinlu-sentiment-analysis")

model_farsi = AutoModelForSeq2SeqLM.from_pretrained("persiannlp/mt5-base-parsinlu-sentiment-analysis")

sent = Sentiment(model_english,tokenizer_english,model_farsi,tokenizer_farsi)
sent.analyze(pd.read_csv('scraped_tweets.csv'))

/Users/emilianopenaloza/mambaforge/envs/weldon/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


tensor([[    0,   375, 34225, 17385,   345,     1]])
tensor([[    0,   375, 34225, 17385,   345,     1]])
tensor([[    0,   259, 32588,     1]])


,date,username,author_id,mentions,hashtags,location,text,sentiment
0,2022-12-05 18:03:58+00:00,RezayatHadis,1551256587129573378,[],"['Iran', 'pianist', 'مهسا_امینی']",République islamique d'Iran,#Iran\n#pianist \n#مهسا_امینی https://t.co/rc2...,no sentiment expressed
1,2022-12-05 17:43:04+00:00,journalistsay,1385266495,[],"['Tehran', 'Iran', 'IranRevolution']",Sadeghiyeh Metro Station | ایستگاه مترو صادقیه,"Dec 5—#Tehran, #Iran \nThe protesters in the c...",positive
2,2022-12-05 11:27:49+00:00,salar__bagheri,1565399154121129986,[],"['اعتصابات_سراسری', 'ایران', 'Iran']",République islamique d'Iran,رییس محترم قوه قضاییه توصیه ای رو به والدین کر...,no sentiment expressed
3,2022-12-04 23:25:24+00:00,alishahbaziart,1236046969140850688,[],"['مهسا_امینی', 'زن_زندگی_آزادی', 'ايران_آزاد',...",République islamique d'Iran,ظلم نمى ماند . . .\n.\n#مهسا_امینی #زن_زندگی_...,negative
4,2022-12-04 21:24:48+00:00,marzprad,1580317621412511744,[],['Iran'],République islamique d'Iran,Wanna know one of many truths related to the I...,positive
5,2022-12-04 14:23:51+00:00,IrnaEnglish,813279088865644544,[],['Iran'],République islamique d'Iran,#Iran FM spox slams crackdown on European prot...,positive
6,2022-12-04 14:19:46+00:00,IrnaEnglish,813279088865644544,[],['Iran'],République islamique d'Iran,Cmdr. says #Iran's Bushehr has progressed in l...,negative
7,2022-12-04 13:48:58+00:00,sheykhian_h,1469389022271262729,[2359926157],"['Iran', 'ChinaUprising']",République islamique d'Iran,Blocking the streets in China is the same meth...,positive
8,2022-12-04 08:39:12+00:00,babakhaghi1982,1410490384484950016,[],"['photography', 'streetphotography', 'outdoors...",République islamique d'Iran,Rainy Afternoon \nhttps://t.co/YIkuOebAZ2\n202...,neutral
9,2022-12-04 08:18:40+00:00,AliGodFather92,830685854544908288,[],"['Europeans', 'protests', 'Iran', 'media']",République islamique d'Iran,Isn't it a bit strange for #Europeans that the...,positive


In [40]:
s2 = re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', ' ',s).replace('\n',' ')
s2 = re.sub(' +', ' ', s2)


'ظلم نمى ماند مهسا امینی زن زندگی آزادی ايران آزاد '

In [39]:
run_model('ظلم نمى ماند مهسا امینی زن زندگی آزادی ايران آزاد')

['negative']


['negative']

Ideally we have the following graphs: 

Number of tweets per hashtag per day 
line chart 
